In [ ]:
#We import necessary libraries.
import time
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import  expected_conditions as ec

#Accept the cookies button and click the fee button.
fees_button = (By.XPATH,"/html/body/div/div[2]/section[2]/div/div/div/div/div/nav/div/div[2]/div[3]/button")
accept_button = (By.XPATH, "/html/body/div/div[2]/div/div[2]/button")

#We use links from excel which we exported from NN IP website and added link column with concat function of excel. You can find it also in the documents.
excel = pd.read_excel('funds-export3.xlsx')
id = excel['Links']

#We add empty columns on our excel files. 
excel['Fees'] = None
excel['AUM'] = None

#For loop for each link to open in chrome and scrap the data.
for i in id:
    driver_path = "C:/Users/user/chromedriver.exe"
    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(i)
    driver.implicitly_wait(7)
    driver.maximize_window()
    wait = WebDriverWait(driver, 45)
    time.sleep(2)
    #Try-except method is for values which we couldn't find and add as a "Not Found" on excel columns.
    try:
        element = wait.until(ec.element_to_be_clickable(accept_button))
        element.click()
        element1 = wait.until(ec.element_to_be_clickable(fees_button))
        element1.click()
        
        #Adding the required javascript id and classes. You can use different id and classes for different website. Inner function helps us to get the last value of class.
        resultfees = driver.execute_script('return document.getElementById("fund-keyfacts__tabs-panel-Fees").getElementsByClassName("DefinitionList_definitionlist__description__1fSov")[0].innerText;')
        resultaum = driver.execute_script('return document.getElementById("fund-keyfacts__tabs-panel-NAV").getElementsByClassName("DefinitionList_definitionlist__description__1fSov")[0].innerText;')
        excel.loc[excel[excel['Links'] == i].index[0], 'Fees'] = resultfees
        excel.loc[excel[excel['Links'] == i].index[0], 'AUM'] = resultaum
        print(resultfees, resultaum)
    except:
        excel.loc[excel[excel['Links'] == i].index[0], 'Fees'] = "Not Found"
        excel.loc[excel[excel['Links'] == i].index[0], 'AUM'] = "Not Found"
        print("Not Found")
    driver.close()
excel.to_excel('funds-export--withfee.xlsx')
